# Assigment 4

In [7]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

In [8]:
def get_data(file="data/irr4-cyl2.off"): 
    v, f = igl.read_triangle_mesh(file)
    tt, _ = igl.triangle_triangle_adjacency(f)
    c = np.loadtxt("data/irr4-cyl2.constraints")
    cf = c[:, 0].astype(np.int64)
    c = c[:, 1:]
    
    return v, f, tt, c, cf

In [9]:
def plot_mesh_field(V, F, R, constrain_faces, col=None, add_lines=True):
    # Highlight in red the constrained faces
    if col is None: 
        col = np.ones_like(F)
        col[constrain_faces, 1:] = 0

    p = mp.plot(V, F, c=col)
    if add_lines:
        # Scaling of the representative vectors
        avg = igl.avg_edge_length(V, F)/2
        #Plot from face barycenters
        B = igl.barycenter(V, F)
        p.add_lines(B, B + R * avg)
    
    return p

# 0. Soft Constraints

In [10]:
def local_basis(V, F):
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :] # first vertex to second vertex
    e2 = V[F[:, 2], :] - V[F[:, 0], :] # first vertex to third vertex

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None] # normalized edge, as x-axis      
    T2 =  np.cross(T1, np.cross(T1, e2)) # perpendicular to the normal and T1
    T2 /= np.linalg.norm(T2, axis=1)[:,None] # normalized
    
    return T1, T2

# in cases like edges, we want the normalized complex representation
# in other cases, we want the complex representation unnormalized such as when we want to compute the energy
def vector_as_complex(vector, basis1, basis2):
    assert(basis1.shape == (3,) and basis2.shape == (3,))
    v = np.array([np.dot(vector, basis1), np.dot(vector, basis2)])
    complex = v[0] + v[1]*1j
    return complex

def common_edge_complex(vector, basis1, basis2):
    assert(basis1.shape == (3,) and basis2.shape == (3,))
    v = np.array([np.dot(vector, basis1), np.dot(vector, basis2)])
    v /= np.linalg.norm(v)
    complex = (v[0] + v[1]*1j).conjugate()
    return complex

def Q_matrix_triplet(V, F, TT, T1, T2):
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]): # Loop over the faces
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite/adjacent face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            ef = common_edge_complex(e, T1[f, :], T2[f, :])
            eg = common_edge_complex(e, T1[g, :], T2[g, :])
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    return data, ii, jj, index

def Q_matrix(data, ii, jj, index, F_shape): 
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F_shape)).asformat("csr")
    return A.H @ A

In [11]:
def align_field_soft(V, F, TT, cf, c, llambda):
    soft_id, soft_value = cf, c
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    T1, T2 = local_basis(V, F)
    data, ii, jj, index = Q_matrix_triplet(V, F, TT, T1, T2)
    
    sqrtl = sqrt(llambda)
    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=complex)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c_projection = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c_projection * sqrtl
        
        index += 1
    
    assert(b.shape[0] == index)
    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    Q = Q_matrix(data, ii, jj, index, F.shape[0])
    u = sp.linalg.spsolve(Q, A.H @ b)
    
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [12]:
v, f, tt, c, cf = get_data()
R = align_field_soft(v, f, tt, cf, c, 1e6)
plot_mesh_field(v, f, R, cf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# 1. Tangent vector fields for scalar field design

## Methods
The main equation simplifies to: 

![](https://latex.codecogs.com/svg.latex?Q_{ff}u_f=-Q_{fc}u_c)<br/>
<!-- $Q_{ff} \tilde{u}_f = -Q_{fc} \tilde{u}_c$ -->

So we now try to find each of the terms in the equation above, solving for `u_f`

In [13]:
# vec \subset U = {0, 1, ..., |universe|-1}
# returns U \ vec
def set_inversion(vec, universe):
    return np.array(sorted(set(range(universe)) - set(vec)))

In [14]:
def get_Q_ff_Q_fc(Q, cf, f_num): 
    # also get non constrained faces
    ncf = set_inversion(cf, f_num)
    
    # Q_ff, Q_fc
    Q_ff = Q[ncf, :][:, ncf]
    Q_fc = Q[ncf, :][:, cf]
    return Q_ff, Q_fc

In [15]:
# T1 and T2 are the local basis vectors at each face
# c is the constraints as 3D vectors, cf is the constrained faces indices
def get_u_c(c, cf, T1, T2): 
    u_c, i = np.zeros(cf.shape[0], dtype=complex), 0
    for f in cf:
        # get the complex representation of the constraint
        complex_c = vector_as_complex(c[i], T1[f, :], T2[f, :])
        u_c[i] = complex_c
        i += 1
    return u_c

In [16]:
def get_cf_to_c_map(cf, c): 
    mp = {}
    for i in range(cf.shape[0]): 
        mp[cf[i]] = c[i]
    return mp

In [17]:
# after getting uf, and having uc and cf, we can get the originally correct u vector
def get_ordered_u(u_f, u_c, cf): 
    u = np.zeros(u_f.shape[0] + u_c.shape[0], dtype=complex)
    cf_set = set(cf)
    u_n = u_f.shape[0] + u_c.shape[0]
    mp = get_cf_to_c_map(cf, u_c)
    for i in range(u_n): 
        # if the current index is in the set of constrained faces, then we need to get the value from u_c
        if i in cf_set:
            u[i] = mp[i]
        else:  
            u[i] = u_f[0]
            u_f = u_f[1:]
    return u

In [18]:
def align_field_hard(V, F, TT, c, cf):
    # get the unmodified Q matrix
    T1, T2 = local_basis(V, F)
    data, ii, jj, index = Q_matrix_triplet(V, F, TT, T1, T2)
    Q = Q_matrix(data, ii, jj, index, F.shape[0])
    
    # now we reorder Q and take Q_ff and Q_fc
    Q_ff, Q_fc = get_Q_ff_Q_fc(Q, cf, F.shape[0])
    
    # then we solve for u_f using the equation that Q_ff * u_f = -Q_fc * u_c
    # solve the linear equation of the hard constraints
    u_c = get_u_c(c, cf, T1, T2)
    neg_Q_fc_u_c = -Q_fc @ u_c
    u_f = sp.linalg.spsolve(Q_ff, neg_Q_fc_u_c)
    
    # re-reorder the resultant u_f into the original order `u`
    u = get_ordered_u(u_f, u_c, cf)
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]
    return R

## Required Output

Visualization of the constraints and interpolated field.

In [19]:
v, f, tt, c, cf = get_data()
R = align_field_hard(v, f, tt, c=c, cf=cf)
plot_mesh_field(v, f, R, cf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

An ASCII dump of the interpolated field (#F x 3 matrix, one vector per row) for the mesh irr4-cyl2.off and the input constraints in the provided file irr4-cyl2.constraints.


In [20]:
# ASCII dump of the mesh and the field
np.savetxt('task1-ascii.txt', R, delimiter=" ", fmt="%s")

# 2. Reconstructing a scalar field from a vector field

## Logic

### Q: Determine the matrix K and vector b in the above minimization (by expanding the least-squares error expression).

ANS: The equation simplifies to minimizing`E = s'*G'*A*G*s-s'*G'*A*u-u'*A*G*s+u'*A*u`. If we place it in terms of the README, we have `K = G'*A*G` [a V x V matrix] and `b = - G'*A*u` [a V x 1 matrix] and `c=u'*A*G*s+u'*A*u` [a scalar]. We then have `E = s'*K*s+2*b'*s+c`.

![](https://latex.codecogs.com/svg.latex?E=s^\top&space;Ks+s^\top&space;b+c)<br/>
<!-- $E = s'^\top Ks - 2b^\top s + c$ -->

![](https://latex.codecogs.com/svg.latex?K=G^\top&space;AG)<br/>
<!-- $K = G'^\top AG$ --> 

![](https://latex.codecogs.com/svg.latex?b=-2G^\top&space;Au)<br/>
<!-- $b = -2G'^\top Au$ -->


where $K$ is a $V \times V$ matrix, $b$ is a $V \times 1$ vector.

<!-- ![](https://latex.codecogs.com/svg.latex?c=u^\top&space;AGs+u^\top&space;Au)<br/> -->
<!-- $c = u'^\top AGs + u'^\top Au$ -->
<!-- is a scalar -->

<!-- $\vec g_t = \nabla S_t = \sum\limits_{\textrm{vertex}~i~\in~t}^3 s_i \nabla\phi_i^t$ -->

### Q: Minimize by differentiating and equating the gradient to zero; this gives you a linear system to solve

ANS: 
We also get `grad E = 2*G'*A*G*s-2*G'*A*u` and we then set it to zero to find the minimum.

This gives the linear sytem `G'*A*G*s = G'*A*u` which we can solve for `s` 

![](https://latex.codecogs.com/svg.latex?\\nabla&space;E=2K\\mathbf{s}+\\mathbf{b})<br/>
<!-- $\nabla E = 2K\mathbf{s} + 2\mathbf{b}$ -->
Setting ![](https://latex.codecogs.com/svg.latex?\\nabla&space;E=0)<br/>
<!-- $\nabla E = 0$ -->
gives the linear system:

![](https://latex.codecogs.com/svg.latex?2K\\mathbf{s}=\\mathbf{-b})<br/>

<!-- $K\mathbf{s} = \mathbf{b}$ -->
Which can be solved for ![](https://latex.codecogs.com/svg.latex?\\mathbf{s})<br/>

<!-- $\mathbf{s}$ -->

so we have the equation 
![](https://latex.codecogs.com/svg.latex?2G^\top&space;AGs=2G^\top&space;Au)<br/>
<!-- $G'^\top AGs = G'^\top Au$ -->

## G, A, and u 

We can simply get G from `igl.grad` and A from `igl.doublearea`.

Because `igl.grad` gives a vertical stack of `G_x, G_y, G_z`, we also need to order A accordingly to stacking it, and also `u` to be a vertical stack of `u_x, u_y, u_z`

In [21]:
# reshapes vertically stacked vector (vx, vy, vz) into horizontally stacked 
def reshape_vertical_xyz_vector(v):
    n = v.shape[0] // 3
    vx, vy, vz = v[:n], v[n:2*n], v[2*n:]
    return np.hstack((vx.reshape(-1, 1), vy.reshape(-1, 1), vz.reshape(-1, 1)))

In [22]:
def get_G(v, f): 
    return igl.grad(v, f)

In [23]:
def get_A(v, f): 
    double_a_vec = igl.doublearea(v, f)
    a_vec = double_a_vec / 2
    a_thrice = np.concatenate([a_vec, a_vec, a_vec])
    A = sp.diags(a_thrice)
    return A

In [24]:
def get_reordered_real_u(R): 
    # u is of size F x 3 (3D vector for each face)
    # we then want to stack the x, y, z components of each face vector
    return np.concatenate([R[:, 0], R[:, 1], R[:, 2]])

In [25]:
def get_G_A_u(v, f, R):
    G = get_G(v, f)
    A = get_A(v, f)
    u = get_reordered_real_u(R)
    return G, A, u

## Solving the linear system

![](https://latex.codecogs.com/svg.latex?K=G^\top&space;AG)<br/>
<!-- $K = G'^\top AG$ -->

![](https://latex.codecogs.com/svg.latex?b=-2G^\top&space;Au)<br/>
<!-- $b = -G'^\top Au$ -->




In [26]:
def get_K(G, A): 
    return G.T @ A @ G

def get_b(G, A, u):
    return - 2 * G.T @ A @ u

In [27]:
# the linear system is not full rank; K has a one dimensional nullspace corresponding to the constant function. This is because a scalar field can be offset by any constant value without altering its gradient. You will need to fix the value at one vertex (e.g., to zero) to solve the system.
# this works for the special case that fixed vertex is 0
def fix_boundary_vertex(K, b, fixed_vertex): 
    K[fixed_vertex, :] = 0
    K[:, fixed_vertex] = 0
    K[fixed_vertex, fixed_vertex] = 1
    b[fixed_vertex] = 0
    return K, b

In [28]:
def reconstruct_scalar_field(A, G, real_u):     
    # get the K matrix and b vector
    K = get_K(G, A)
    b = get_b(G, A, real_u)
    
    # fix the boundary vertex
    K, b = fix_boundary_vertex(K, b, 0)
    two_K = K * 2

    # results 
    s = sp.linalg.spsolve(two_K, -b)
    return s

## Poisson Reconstruction Error

In [29]:
def get_error(G, s, real_u):
    g = G @ s
    a = g - real_u
    a2 = reshape_vertical_xyz_vector(a)
    return np.linalg.norm(a2, axis=1)

## Plotting the scalar field

In [30]:
def task_2_plots(v, f, cf, R, s, error):
    print("PLOT VECTOR FIELD")
    plot_mesh_field(v, f, R, cf)
    print("PLOT SCALAR FIELD")
    plot_mesh_field(v, f, R, cf, col=s)
    print("PLOT POISSON ERROR")
    plot_mesh_field(v, f, R, cf, col=error)     
    

def task_2():
    v, f, tt, c, cf = get_data()
    R = align_field_hard(v, f, tt, c=c, cf=cf)
    G, A, real_u = get_G_A_u(v, f, R)
    s = reconstruct_scalar_field(A, G, real_u)
    error = get_error(G, s, real_u)
    task_2_plots(v, f, cf, R, s, error)
    return s

## Required Output

Visualization of computed scalar function and its gradient.
Plots of the Poisson reconstruction error.

In [31]:
s = task_2()

PLOT VECTOR FIELD


/Users/brayton/opt/anaconda3/envs/my_tf_env/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

PLOT SCALAR FIELD


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

PLOT POISSON ERROR


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

An ASCII dump of the reconstructed scalar function (#V x 1 vector one vertex value per row) for the mesh irr4-cyl2.off and the input constraints in irr4-cyl2.constraints.

In [32]:
np.savetxt('task2-ascii.txt', s, delimiter=" ", fmt="%s")

# 3. Harmonic and LSCM Parameterization

## Gradient

In [33]:
# returns gradient w.r.t u and v
def get_mapping_gradient(v, f, uv): 
    G = get_G(v, f)
    g_u_vertical = G @ uv[:, 0]
    g_v_vertical = G @ uv[:, 1]
    g_u, g_v = reshape_vertical_xyz_vector(g_u_vertical), reshape_vertical_xyz_vector(g_v_vertical)
    return g_u, g_v

## Plotting Utility

In [34]:
def plot_subplots(v, f, uv): 
    p = mp.subplot(v, f, uv=uv, s=[1, 2, 0])
    mp.subplot(uv, f, uv=uv, shading={"wireframe": True}, data=p, s=[1, 2, 1])

In [35]:
def plot_gradient(v, f, uv): 
    avg = igl.avg_edge_length(v, f)/2
    centers = igl.barycenter(v, f)
    g_u, g_v = get_mapping_gradient(v, f, uv)
    p2 = mp.plot(v, f, uv=uv)
    p2.add_lines(centers, centers + g_u * avg)
    p2.add_lines(centers, centers + g_v * avg)

In [36]:
def plot_parametization(v, f, uv, show_gradient=False, show_subplots=False): 
    if show_subplots: 
        plot_subplots(v, f, uv)
    
    # add lines to show gradient
    if show_gradient:
        plot_gradient(v, f, uv)

## Harmonic

In [37]:
# background parameter
def harmonic(v, f): 
    # you will first map the mesh boundary to a unit circle in the UV plane centered at the origin.
    bnd = igl.boundary_loop(f) # Compute ordered boundary loops for a manifold mesh and return the longest loop in terms of vertices.
    uv_bc = igl.map_vertices_to_circle(v, bnd)
    
    # The boundary U and V coordinates are then "harmonically interpolated" into the interior
    uv = igl.harmonic(v, f, bnd, uv_bc, 1)
    return uv

## LSCM

In [38]:
def lscm(v, f): 
    # Fix two arbitrary boundary vertices at (0, 1) and (1, 1)
    bnd = igl.boundary_loop(f)
    bnd_vertices = np.array([bnd[0], bnd[1]])
    bnd_uv = np.array([[0.0,0.0],[0.0,1.0]])
    
    # Compute LSCM bnd[1]
    _, uv = igl.lscm(v, f, bnd_vertices, bnd_uv)
    return uv

## Required Output

Visualization of the computed mapping functions and their gradients for LSCM and harmonic mapping.

Harmonic

In [39]:
file = "data/camel_head.off"
v, f, _, _, _ = get_data(file)
uv = harmonic(v, f)

In [40]:
plot_subplots(v, f, uv)

In [41]:
plot_gradient(v, f, uv)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

LSCM

In [42]:
file = "data/camel_head.off"
v, f, _, _, _ = get_data(file)
uv = lscm(v, f)

In [43]:
plot_subplots(v, f, uv)

In [44]:
plot_gradient(v, f, uv)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

# 4. Editing a parameterization with vector fields

## Editing the parameterization

We fix either U or V and then solve for the other by using constraints. This can be helpful for making a better parameterization.

In [45]:
def task_4_uv(): 
    # step 0 
    v, f, tt, c, cf = get_data()

    # step 1 
    uv = harmonic(v, f)

    # step 2 
    R = align_field_hard(v, f, tt, c=c, cf=cf)

    # step 3 
    G, A, real_u = get_G_A_u(v, f, R)
    s = reconstruct_scalar_field(A, G, real_u)

    # we fix U and change V
    uv_reconstructed = np.column_stack([uv[:,0], s])
    return uv_reconstructed, v, f, cf

## Detecting problems with the parameterization

Each 3D vertex maps to a 2D UV point. Our goal is to loop through each triangle in the UV plane and check if the signed area is negative (or look into the sign of the normal to each triangle). If it is, it is flipped. To do so, we look into the vertex in the UV array as the points.

In [46]:
# (u, v, 0)
def uv_to_3d(uv_point):
    return np.array([uv_point[0], uv_point[1], 0])

def get_flipped_triangles(f, uv):
    flipped = []
    # for each triangle
    for face in range(f.shape[0]): 
        # get the vertices' indices
        vidx1, vidx2, vidx3 = f[face]
        # get the UV coordinates as now 3D points 
        v1, v2, v3 = uv_to_3d(uv[vidx1]), uv_to_3d(uv[vidx2]), uv_to_3d(uv[vidx3])
        # get the normal of the triangle
        normal = np.cross(v2 - v1, v3 - v1)
        # if negative, then the triangle is flipped
        if normal[2] < 0: 
            flipped.append(face)
    # we want the "flipped" triangles to be the minority flip
    if len(flipped) > len(f) / 2:  
        flipped = set_inversion(flipped, len(f))
    return flipped

In [47]:
def plot_flipped_triangles(flipped, v, f, cf): 
    color = np.zeros_like(f)
    color[flipped, 1:] = 1
    plot_mesh_field(v, f, np.zeros_like(v), cf, col=color, add_lines=False)

## Required Output

In [48]:
uv_reconstructed, v, f, cf = task_4_uv()
flipped_triangles = get_flipped_triangles(f, uv_reconstructed)

Visualization of the edited parameterization.

In [49]:
plot_subplots(v, f, uv_reconstructed)

Visualization of flipped elements.

In [50]:
plot_flipped_triangles(flipped_triangles, v, f, cf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

An ASCII dump of the flipped triangle indices (if any) resulting from an edited harmonic parameterization of the mesh irr4-cyl2.off, where the parameterization's V coordinate is replaced with a scalar field designed from the gradient vector constraints provided in irr4-cyl2.constraints.

In [51]:
np.savetxt('task4.txt', flipped_triangles, delimiter=" ", fmt="%s")